# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key
from config import api_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
file= '../Weather_Jake.py/weather_data.csv'
weather_data_df= pd.read_csv(file)

weather_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ancud,-41.8697,-73.8203,55.99,94,90,17.27,CL,1619813295
1,talcahuano,-36.7167,-73.1167,60.80,77,90,17.27,CL,1619813295
2,okhotsk,59.3833,143.3000,30.69,80,100,4.65,RU,1619813295
3,ushuaia,-54.8000,-68.3000,41.00,81,75,4.61,AR,1619813261
4,saint-philippe,-21.3585,55.7679,75.20,83,0,5.75,RE,1619813296
...,...,...,...,...,...,...,...,...,...
537,kailua,21.4022,-157.7394,80.60,73,90,11.50,US,1619813405
538,ahmadpur,18.7000,76.9333,86.56,31,69,7.34,IN,1619813405
539,road town,18.4167,-64.6167,82.40,65,40,10.36,VG,1619813149
540,dedza,-14.3779,34.3332,58.57,88,5,1.97,MW,1619813405


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
#capture lat and long
locations= weather_data_df[['Lat', 'Lng']]

humidity_weight= weather_data_df['Humidity']


In [23]:
#plot heatmap with gmaps
fig = gmaps.figure()

#create heat layer on top of gmaps
#make weight of heatmap reflect humidity
heat_layer= gmaps.heatmap_layer(locations, weights=humidity_weight, dissipating=False, max_intensity=100,
                                 point_radius=1.5)

#add layer to figure
fig.add_layer(heat_layer)

#Display combined
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
#reference main df
weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ancud,-41.8697,-73.8203,55.99,94,90,17.27,CL,1619813295
1,talcahuano,-36.7167,-73.1167,60.80,77,90,17.27,CL,1619813295
2,okhotsk,59.3833,143.3000,30.69,80,100,4.65,RU,1619813295
3,ushuaia,-54.8000,-68.3000,41.00,81,75,4.61,AR,1619813261
4,saint-philippe,-21.3585,55.7679,75.20,83,0,5.75,RE,1619813296


In [25]:
hotel_df =weather_data_df.loc[(weather_data_df['Max Temp'] > 65) &
                    (weather_data_df['Max Temp'] < 87) &
                    (weather_data_df['Wind Speed'] < 7) &
                    (weather_data_df['Cloudiness'] < 10)]

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,saint-philippe,-21.3585,55.7679,75.20,83,0,5.75,RE,1619813296
18,netrakona,24.8833,90.7167,79.65,50,4,5.32,BD,1619813298
47,umm lajj,25.0213,37.2685,85.86,38,0,6.51,SA,1619813303
83,campos novos,-27.4017,-51.2250,65.34,52,5,4.63,BR,1619813310
97,trinidad,-14.8333,-64.9000,84.20,39,0,6.91,BO,1619813041
136,jacareacanga,-6.2222,-57.7528,82.40,78,0,4.61,BR,1619813320
151,doha,25.2867,51.5333,82.99,65,0,5.75,QA,1619813239
169,saint-joseph,-21.3667,55.6167,75.20,83,0,5.75,RE,1619813326
178,ampanihy,-24.7000,44.7500,70.88,71,0,6.91,MG,1619813329
191,yichang,30.7144,111.2847,69.62,70,0,3.91,CN,1619813331


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#reference
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,saint-philippe,-21.3585,55.7679,75.20,83,0,5.75,RE,1619813296
18,netrakona,24.8833,90.7167,79.65,50,4,5.32,BD,1619813298
47,umm lajj,25.0213,37.2685,85.86,38,0,6.51,SA,1619813303
83,campos novos,-27.4017,-51.2250,65.34,52,5,4.63,BR,1619813310
97,trinidad,-14.8333,-64.9000,84.20,39,0,6.91,BO,1619813041


In [27]:
hotel_df['Hotel Name']= ''
hotel_df.head()

C:\Users\burns\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,saint-philippe,-21.3585,55.7679,75.20,83,0,5.75,RE,1619813296,
18,netrakona,24.8833,90.7167,79.65,50,4,5.32,BD,1619813298,
47,umm lajj,25.0213,37.2685,85.86,38,0,6.51,SA,1619813303,
83,campos novos,-27.4017,-51.2250,65.34,52,5,4.63,BR,1619813310,
97,trinidad,-14.8333,-64.9000,84.20,39,0,6.91,BO,1619813041,


In [28]:
#call using Google Places API

#https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
#output is json  so:

#base url to be used later:   https://maps.googleapis.com/maps/api/place/nearbysearch/json (parameters)

#API DOC SITE: https://developers.google.com/maps/documentation/places/web-service/search#PlaceSearchRequests

In [29]:
# set base_url to be used with parameters

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [30]:
#api call requires parameters to be set up

# params dictionary to update each iteration
paramaters = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

###### Test API Call URL:

In [31]:
#test url

#don't print with api key on public repository

#         get lat, lng from df
# lat_test = -21.3585
# lng_test = 55.7679


# paramaters['location']= f"{lat_test},{lng_test}"

#           make request and print url
# name_address = requests.get(base_url, params=paramaters)
    
#          print the name_address url, avoid doing for public github repos in order to avoid exposing key
#print(name_address.url)


In [32]:
#create test df of nly 5 cities to test when making first call to ensure code works


#this allows you to check code without making too many pulls

test_df = hotel_df.head(1)
test_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,saint-philippe,-21.3585,55.7679,75.2,83,0,5.75,RE,1619813296,


#### Once code is confirmed to work, replace test_df  with  hotel_df and collect hotel name for all cities:

In [33]:
# use for loop with api call to obtain all cities in df using hotel_df

# .index neaded to grab value at df location

for near_hotel in hotel_df.index:
    # get lat, lng from df
    #iterate through each row(near_hotel) 
    lat = hotel_df["Lat"][near_hotel]
    lng = hotel_df["Lng"][near_hotel]
    
    # change location each iteration while leaving original params in place
    #adds to existing parameter dictionary
    paramaters["location"] = f"{lat},{lng}"
    
    #request url and convert to json
    #request_hotel= (requests.get(base_url, params=paramaters))
    request_json_hotel= (requests.get(base_url, params=paramaters)).json()
    

    #look at json for rfeference
#print(json.dumps(request_json_hotel, indent=4))
    
    
    
    #use try and except to keep code running even if data is missing
    
    #include near_hotel as the index for .loc
    try:
        hotel_df.loc[near_hotel, 'Hotel Name'] = request_json_hotel['results'][0]['name']
        
    except (KeyError, IndexError):
        print('Data Error.....skipping.')
    



C:\Users\burns\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Data Error.....skipping.
Data Error.....skipping.
Data Error.....skipping.


In [34]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,saint-philippe,-21.3585,55.7679,75.20,83,0,5.75,RE,1619813296,"Chambres d'hôte ""La Trinité"""
18,netrakona,24.8833,90.7167,79.65,50,4,5.32,BD,1619813298,kurpar gas office
47,umm lajj,25.0213,37.2685,85.86,38,0,6.51,SA,1619813303,Moon light Furnished Units
83,campos novos,-27.4017,-51.2250,65.34,52,5,4.63,BR,1619813310,Hotel Corujão - Campos Novos
97,trinidad,-14.8333,-64.9000,84.20,39,0,6.91,BO,1619813041,Hotel Piesta
136,jacareacanga,-6.2222,-57.7528,82.40,78,0,4.61,BR,1619813320,Casa do Ian😎
151,doha,25.2867,51.5333,82.99,65,0,5.75,QA,1619813239,Movenpick Hotel Doha
169,saint-joseph,-21.3667,55.6167,75.20,83,0,5.75,RE,1619813326,"""Plantation Bed and Breakfast"
178,ampanihy,-24.7000,44.7500,70.88,71,0,6.91,MG,1619813329,Angora Hotel
191,yichang,30.7144,111.2847,69.62,70,0,3.91,CN,1619813331,Gezhouba Hotel


In [46]:
clean_hotel_df = hotel_df.dropna(subset= ['Hotel Name'])
clean_hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,saint-philippe,-21.3585,55.7679,75.20,83,0,5.75,RE,1619813296,"Chambres d'hôte ""La Trinité"""
18,netrakona,24.8833,90.7167,79.65,50,4,5.32,BD,1619813298,kurpar gas office
47,umm lajj,25.0213,37.2685,85.86,38,0,6.51,SA,1619813303,Moon light Furnished Units
83,campos novos,-27.4017,-51.2250,65.34,52,5,4.63,BR,1619813310,Hotel Corujão - Campos Novos
97,trinidad,-14.8333,-64.9000,84.20,39,0,6.91,BO,1619813041,Hotel Piesta
136,jacareacanga,-6.2222,-57.7528,82.40,78,0,4.61,BR,1619813320,Casa do Ian😎
151,doha,25.2867,51.5333,82.99,65,0,5.75,QA,1619813239,Movenpick Hotel Doha
169,saint-joseph,-21.3667,55.6167,75.20,83,0,5.75,RE,1619813326,"""Plantation Bed and Breakfast"
178,ampanihy,-24.7000,44.7500,70.88,71,0,6.91,MG,1619813329,Angora Hotel
191,yichang,30.7144,111.2847,69.62,70,0,3.91,CN,1619813331,Gezhouba Hotel


In [47]:
clean_hotel_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
Hotel Name     object
dtype: object

In [39]:



# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = '''
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
'''
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]






Figure(layout=FigureLayout(height='420px'))

In [44]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations,info_box_content=hotel_info, display_info_box=True)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))